In [21]:
import requests, time,shlex,subprocess
import mimetypes as mim
import base64
from matplotlib.pyplot import imread
import re
page = 1
res =[]
from PIL import Image,ImageFile
import numpy as np
ImageFile.LOAD_TRUNCATED_IMAGES = True
!pip3 install pytesseract
!pip3 install  Gitpython
import pytesseract
import git
import os

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
req = requests.get('https://api.github.com/search/repositories?q=microservice+stars%3A521&type=Repositories&ref=advsearch&l=&l=+str(page)',auth=('gess18','3de21fec1f9d02f5f573c7bf9224cc8f22c6abeb'))
res = req.json()
print(res)

In [ ]:
#search for contents
def check_text(typeGuess):
    if(re.search(typeGuess[0],'text') or re.search(typeGuess[1],'json') or re.search(typeGuess[1],'javascript')):
         return True
    else:
         return False

In [ ]:
#get type and check if image
def check_img(typeGuess):
    print()
    if(re.search(typeGuess[0],'image')):
         return True
    else:
         return False

In [ ]:

    repos=res['items']
    print(len(repos))
    for repoIndx in range(0, len(repos)):
            repoName=repos[repoIndx]['full_name'].split('/')[1]
            #We will the contents link for repo by using 'content_url' key.
            gitsUrl = repos[repoIndx]['git_url']
            print(gitsUrl.split(':')[1])
            if(os.path.isdir(repos[repoIndx]['full_name'].split('/')[-2])==False):
                       os.mkdir(repos[repoIndx]['full_name'].split('/')[-2])
            print(repos[repoIndx]['full_name'].split('/')[-2])    
            git.Git(repos[repoIndx]['full_name'].split('/')[-2]).clone("https:"+gitsUrl.split(':')[1])

In [ ]:
"""
while (res['items'] != []):
    #print(res)
    time.sleep(10)
    total_count =res['total_count']
    repoDic={}
    repos=res['items']
    for repoIndx in range(0, len(repos)):
            repoName=repos[repoIndx]['full_name'].split('/')[1]
            repoDic[repoName] = {}
            repoDic[repoName]['desc']=repos[repoIndx]['description']
            #We will the contents link for repo by using 'content_url' key.
            contentsUrl = repos[repoIndx]['contents_url']   
            contentsUrl=contentsUrl.split('{+path}')[0]
            #Sending the request to get folders and files 
            gitReq=requests.get(contentsUrl,auth=('gess18','3de21fec1f9d02f5f573c7bf9224cc8f22c6abeb'))
            contents=gitReq.json()
            time.sleep(5)
            for contentIndx in range(0,len(contents)):
                    #print(contents)
                    # Guessing the extensions of the files by mimtypes.
                    typeGuess = mim.guess_type(contents[contentIndx]['name'])
                    if(typeGuess[0] is not None):
                            typeGuess=typeGuess[0].split('/')
                            typeGuessImg=check_img(typeGuess)
                            #print(typeGuessImg)
                            typeGuess=check_text(typeGuess) or check_img(typeGuess)
                    elif(contents[contentIndx]['type']=='file' and (re.search(contents[contentIndx]['name'],'description') or contents[contentIndx]['name']=='README')):
                            typeGuess = typeGuess[0]
                            gitReq=requests.get(contents[contentIndx]['git_url'],auth=('gess18','3de21fec1f9d02f5f573c7bf9224cc8f22c6abeb'))
                            fileContent=gitReq.json()
                            fileName=contents[contentIndx]['name']
                            fileBytes=base64.b64decode(fileContent['content'])
                            repoDic[repoName][fileName]=fileBytes.decode('utf-8')
                            time.sleep(3)
                            continue
                    #print(typeGuess)        
                    if(contents[contentIndx]['type']=='file' and typeGuess==True ):
                        gitReq=requests.get(contents[contentIndx]['git_url'],auth=('gess18','3de21fec1f9d02f5f573c7bf9224cc8f22c6abeb'))
                        fileName=contents[contentIndx]['name']
                        fileContent=gitReq.json()
                        fileBytes=base64.b64decode(fileContent['content'])
                        #print(typeGuessImg)
                        if(typeGuessImg):  
                                #print(fileBytes)
                                if(fileName.split('.')[-1] == 'SVG' or  fileName.split('.')[-1] == 'svg'):
                                           continue
                                imgFile=open('ImgToSave.'+fileName.split('.')[-1],'wb')
                                #imgFile=open(fileName)
                                imgFile.write(fileBytes)
                                imgFile.close()
                                #Extract the text from the image.
                                '''
                                imgFile=Image.open(fileName,'r').convert('RGB')
                                pixels=np.asarray(imgFile,np.uint8)
                                print(pixels.shape)
                                convertedImg=abs(np.fft.rfft2(pixels,axes=(0,1)))
                                print(convertedImg.shape)
                                convertedImg=np.uint8(convertedImg)
                                imgFile=Image.fromarray(convertedImg)
                                imgFile.save(fileName)
                                '''    
                                if(fileName.split('.')[-1]=='png'or fileName.split('.')[-1]=='PNG'):
                                            imgFile=Image.open('ImgToSave.'+fileName.split('.')[-1],'r').convert('RGB')
                                            imgFile.save('ImgToSave.jpg')
                                            fileName='ImgToSave.jpg'
                                textOfImg=pytesseract.image_to_string(imread('ImgToSave.'+fileName.split('.')[-1]))
                                repoDic[repoName][fileName]=textOfImg
                        else:
                                repoDic[repoName][fileName]=fileBytes.decode('utf-8')
                                  
                    elif(contents[contentIndx]['type']=='dir'): 
                                dirContents=requests.get(contents[contentIndx]['git_url']+'?recursive=-1',auth=('gess18','3de21fec1f9d02f5f573c7bf9224cc8f22c6abeb'))
                                files= dirContents.json()['tree']
                                #print(files)
                                time.sleep(3)
                                for fileIndx in range(len(files)):
                                        #print(files[fileIndx])
                                        if(files[fileIndx]['type']=='tree'):
                                            continue
                                        typeGuess = mim.guess_type(files[fileIndx]['path'])
                                        if(typeGuess[0] is not None):
                                            typeGuess=typeGuess[0].split('/')
                                            #print(typeGuess)
                                            typeGuessImg=check_img(typeGuess)    
                                            typeGuess= check_text(typeGuess) or check_img(typeGuess)
                                           
                                        elif(files[fileIndx]['type']=='blob' and (re.search(files[fileIndx]['path'],'description') or files[fileIndx]['path'].split('/')[-1]=='README')):
                                            typeGuess=typeGuess[0]
                                            fileContent=requests.get(files[fileIndx]['url'],auth=('gess18','3de21fec1f9d02f5f573c7bf9224cc8f22c6abeb'))
                                            fileName=files[fileIndx]['path'].split('/')[-1]
                                            repoDic[repoName][fileName]=fileBytes.decode('utf-8')
                                            time.sleep(3)
                                            continue
                                        print(files[fileIndx])    
                                        if(files[fileIndx]['type']=='blob' and typeGuess==True):
                                                fileContent=requests.get(files[fileIndx]['url'],auth=('gess18','3de21fec1f9d02f5f573c7bf9224cc8f22c6abeb')) 
                                                fileContent=fileContent.json()
                                                #print(fileContent)
                                                fileName=files[fileIndx]['path'].split('/')[-1]
                                                fileBytes=base64.b64decode(fileContent['content']) 
                                                #print(typeGuessImg)
                                                if(typeGuessImg):  
                                                    #print(fileBytes)
                                                    if(fileName.split('.')[-1] == 'SVG' or  fileName.split('.')[-1] == 'svg'):
                                                             continue
                                                    imgFile=open('ImgToSave.'+fileName.split('.')[-1],'wb')
                                                    imgFile.write(fileBytes)
                                                    imgFile.close()
                                                    #Extract the text from the image.
                                                    '''                                                    
                                                    imgFile=Image.open(fileName,'r').convert('RGB')
                                                    pixels=np.asarray(imgFile,np.uint8)
                                                    print(pixels.shape)
                                                    convertedImg=abs(np.fft.rfft2(pixels,axes=(0,1)))
                                                    print(convertedImg.shape)
                                                    convertedImg=np.uint8(convertedImg)
                                                    imgFile=Image.fromarray(convertedImg)
                                                    imgFile.save(fileName)
                                                    '''    
                                                    if(fileName.split('.')[-1]=='png'or fileName.split('.')[-1]=='PNG'):
                                                            imgFile=Image.open('ImgToSave.'+fileName.split('.')[-1],'r').convert('RGB')
                                                            imgFile.save('ImgToSave.jpg')
                                                            fileName='ImgToSave.jpg'
                                                            
                                                    textOfImg=pytesseract.image_to_string(imread('ImgToSave.'+fileName.split('.')[-1]))
                                                    
                                                    repoDic[repoName][fileName]=textOfImg
                                                else:
                                                    repoDic[repoName][fileName]=fileBytes.decode('utf-8')
                                                    
                                                time.sleep(12)
            repoFile = open(repoName,'w')
            repoFile.write(str(repoDic))
            repoFile.close()
            repoDic={}
    page +=1
    req = requests.get('https://api.github.com/search/repositories?q=microservice+stars%3A521&type=Repositories&ref=advsearch&l=&l=+str(page)',auth=('gess18','3de21fec1f9d02f5f573c7bf9224cc8f22c6abeb'))
    res = req.json()      
    first=0
""""

# Display Microservice repo contents

In [22]:
path = "/Users/gleschen/Downloads/test_raw_file.txt"
file=open(path,'r')
print(file.readlines())

['*\\n *  or in the "license" file accompanying this file. This file is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES *\\{\'iot-device-simulator\': {\'desc\': \'The IoT Device Simulator solution is a Graphical User Interface (GUI) based engine designed to enable customers to get started quickly assessing AWS IoT services without an existing pool of devices. The IoT Device Simulator helps effortlessly create and simulate thousands of connected devices that are defined by the customer.\', \'app.js\': \'/*********************************************************************************************************************\\n *  Copyright 2016 Amazon.com, Inc. or its affiliates. All Rights Reserved.                                           *\\n *                                                                                                                    *\\n *  Licensed under the Amazon Software License (the "License"). You may not use this file except in compliance        *\\n 

# Data cleaning

In [23]:
path = "/Users/gleschen/Downloads/test_raw_file.txt"
file=open(path,'r')
lines=file.readlines()
cummulateString=''
for line in lines:
#print(newLine)
    for char in line:
# Exclude the non alphabitical data.
        if(char.isalpha()):
            cummulateString+=char
        elif(char.isspace()):
            cummulateString+=''
        elif(char.isupper()):
            char=char.lower()
            cummulateString+=char
        else:
            cummulateString+=' '
            cummulateString+=' '
print(cummulateString)
#file.close()
cummulateString+=' '

    n  orinthe  license  fileaccompanyingthisfile  Thisfileisdistributedonan  ASIS  BASIS  WITHOUTWARRANTIES        iot  device  simulator        desc      TheIoTDeviceSimulatorsolutionisaGraphicalUserInterface  GUI  basedenginedesignedtoenablecustomerstogetstartedquicklyassessingAWSIoTserviceswithoutanexistingpoolofdevices  TheIoTDeviceSimulatorhelpseffortlesslycreateandsimulatethousandsofconnecteddevicesthataredefinedbythecustomer        app  js                                                                                                                                                                                                                                                    n  Copyright        Amazon  com  Inc  oritsaffiliates  AllRightsReserved      n      n  LicensedundertheAmazonSoftwareLicense  the  License      Youmaynotusethisfileexceptincompliance    n  withtheLicense  AcopyoftheLicenseislocatedat    n      n  http      aws  amazon  com  asl      n  n  ORCONDITIONSOF

In [24]:
#import nltk
#nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
set(stopwords.words('english'))

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [25]:
import sys
'''
def remove_stopwords(cummulateString):
        processed_word_list = []
        for word in cummulateString:
            word = word.lower() # in case they aren't all lower case
            if word not in stopwords.words("english"):
                processed_word_list.append(word)
        print (cummulateString)
        return processed_word_list
'''
#example_sent = "This is a sample sentence, showing off the stop words filtration."
  
stop_words = set(stopwords.words('english')) 
word_tokens = word_tokenize(cummulateString)  
filtered_sentence = [w for w in word_tokens if not w in stop_words]  
filtered_sentence = [] 
for w in word_tokens: 
    if w not in stop_words: 
        filtered_sentence.append(w) 
print(word_tokens) 
print(filtered_sentence) 
with open('cleaned_data.txt', 'w') as f:
    print(filtered_sentence, file=f)  
file.close()

['n', 'orinthe', 'license', 'fileaccompanyingthisfile', 'Thisfileisdistributedonan', 'ASIS', 'BASIS', 'WITHOUTWARRANTIES', 'iot', 'device', 'simulator', 'desc', 'TheIoTDeviceSimulatorsolutionisaGraphicalUserInterface', 'GUI', 'basedenginedesignedtoenablecustomerstogetstartedquicklyassessingAWSIoTserviceswithoutanexistingpoolofdevices', 'TheIoTDeviceSimulatorhelpseffortlesslycreateandsimulatethousandsofconnecteddevicesthataredefinedbythecustomer', 'app', 'js', 'n', 'Copyright', 'Amazon', 'com', 'Inc', 'oritsaffiliates', 'AllRightsReserved', 'n', 'n', 'LicensedundertheAmazonSoftwareLicense', 'the', 'License', 'Youmaynotusethisfileexceptincompliance', 'n', 'withtheLicense', 'AcopyoftheLicenseislocatedat', 'n', 'n', 'http', 'aws', 'amazon', 'com', 'asl', 'n', 'n', 'ORCONDITIONSOFANYKIND', 'expressorimplied', 'SeetheLicenseforthespecificlanguagegoverningpermissions', 'n', 'andlimitationsundertheLicense', 'n', 'n', 'n', 'n', 'authorSolutionBuilders', 'n', 'n', 'n', 'usestrict', 'n', 'nconstf

# Searching Contents for Keywords

In [ ]:
'''
import mmap
with open('cleaned_data.txt') as f:
    s = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)
    if s.find(b'IoT'or b'device' or b'automotive' or b'vehicle' or b'car') != -1:
        print("Microservice is classified as: Automotive")
'''

In [26]:
import pandas
df = pandas.read_csv('/Users/gleschen/Downloads/dictionary.csv')
df = df.fillna('no')
print(df)

            Banking Automotive    Media  Fintech  Transportation  Law  \
0              bank        iot  netflix  fintech  transportation  law   
1          mortgage     device       no       no              no   no   
2  checking account         no       no       no              no   no   

  Food&Beverages Healthcare Education   Real Estate  Insurance       Energy  \
0       beverage     health    school  construction  insurance  electricity   
1             no         no        no            no         no           no   
2             no         no        no            no         no           no   

   Retail  Encryption  Monitoring  Search  Authentication  Logging  Upload  \
0  retail  encryption        siem  search  authentication  logging  upload   
1      no  decryption  monitoring      no              no       no      no   
2      no          no          no      no              no       no      no   

  Other  
0    no  
1    no  
2    no  


In [ ]:
print(list(df.values))

In [ ]:
'''
import csv
#reader = csv.DictReader(open("file2.csv"))
df = csv.DictReader(open("/Users/gleschen/Downloads/dictionary.csv"))
for raw in df:
    orderedDict = raw
    #print(raw)
    print (raw)
'''

In [ ]:
#print(orderedDict.values())

In [27]:
import mmap
with open('/Users/gleschen/Downloads/cleaned_data.txt') as f:
    s = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)
    dic = orderedDict.values()
    print(list(dic))
    d = list(dic)
    #if s.find(b'dic') != -1:
    print(s.find(bytes(d[1].encode('utf-8'))))
    #print("Microservice is classified as: Automotive")
    f.close()

NameError: name 'orderedDict' is not defined

# Classification

In [28]:
with open('/Users/gleschen/Downloads/_huge_Java-Bank-Project.txt') as f:
    cleanedData=f.read()
    for col in df.columns:
        for indx in range(0,len(df.loc[:,col])):
          if df.loc[:,col][indx] in cleanedData:
            #print(key.d[0])
            #print([key for key in raw.keys()][0])
            print("This microservice is classified as:", col)
            #print("true")

This microservice is classified as: Banking
This microservice is classified as: Banking
This microservice is classified as: Banking
This microservice is classified as: Automotive
This microservice is classified as: Automotive
This microservice is classified as: Automotive
This microservice is classified as: Media
This microservice is classified as: Media
This microservice is classified as: Fintech
This microservice is classified as: Fintech
This microservice is classified as: Transportation
This microservice is classified as: Transportation
This microservice is classified as: Law
This microservice is classified as: Law
This microservice is classified as: Food&Beverages
This microservice is classified as: Food&Beverages
This microservice is classified as: Healthcare
This microservice is classified as: Healthcare
This microservice is classified as: Education
This microservice is classified as: Education
This microservice is classified as: Real Estate
This microservice is classified as: R

In [ ]:
""""
import ast
countDic ={}
from keras.preprocessing.text import Tokenizer,text_to_word_sequence
with open('/Users/gleschen/Downloads/cleaned_data.txt') as f:
    cleanedData=f.read()
    for col in df.columns:
        countDic[col]={}
        for indx in range(0,len(df.loc[:,col])):         
          print(df.loc[:,col][indx])
          if df.loc[:,col][indx] in cleanedData and df.loc[:,col][indx]!='no':
            countDic[col] +=1    
            cleanedatalist = ast.literal_eval(cleanedData)
            t=Tokenizer()
            t.fit_on_texts(cleanedatalist)
            print("Keyword:", df.loc[:,col][indx], "\nOccurrence:", t.word_counts[df.loc[:,col][indx]])
            #print(text_to_word_sequence(cleanedData))
            #print(key.d[0])
            #print([key for key in raw.keys()][0])
            print("This microservice is classified as:", col)
            #print("true")
"""

In [30]:
import ast
countDic ={}
countDic['Topic']={}
countDic['Topic']['H']={}
countDic['Topic']['V']={}
type_1='V'
from keras.preprocessing.text import Tokenizer,text_to_word_sequence
with open('/Users/gleschen/Downloads/_huge_Java-Bank-Project') as f:
    cleanedData=f.read()
    for indxCol in range(0,len(df.columns)):
        if(indxCol < 13):
            #print(df.columns[indxCol])
            countDic['Topic'][type_1][df.columns[indxCol]]=0
        else:
            type_1='H'
            countDic['Topic'][type_1][df.columns[indxCol]]=0
        for indx in range(0,len(df.loc[:,df.columns[indxCol]])): 
          #print(df.loc[:,df.columns[indxCol]][indx])
          if df.loc[:,df.columns[indxCol]][indx] in cleanedData and df.loc[:,df.columns[indxCol]][indx]!='no':
            countDic['Topic'][type_1][df.columns[indxCol]]+=1
            #print(countDic['Topic'][type_1][df.columns[indxCol]])
            print(df.loc[:,df.columns[indxCol]][indx])
            cleanedatalist = ast.literal_eval(cleanedData)
            t=Tokenizer()
            t.fit_on_texts(cleanedatalist)
            #print("Keyword:", df.loc[:,df.columns[indxCol]][indx], "\nOccurrence:", t.word_counts[df.loc[:,df.columns[indxCol]][indx]])
            #print(text_to_word_sequence(cleanedData))
            #print(key.d[0])
            #print([key for key in raw.keys()][0])
            #print("This microservice is classified as:", df.columns[indxCol])
            #print("true")countDic['horizontal']={}

bank
device
search
authentication
logging
upload


In [31]:
countDic

{'Topic': {'H': {'Encryption': 0,
   'Monitoring': 0,
   'Search': 1,
   'Authentication': 1,
   'Logging': 1,
   'Upload': 1,
   'Other': 0},
  'V': {'Banking': 1,
   'Automotive': 1,
   'Media': 0,
   'Fintech': 0,
   'Transportation': 0,
   'Law': 0,
   'Food&Beverages': 0,
   'Healthcare': 0,
   'Education': 0,
   'Real Estate': 0,
   'Insurance': 0,
   'Energy': 0,
   'Retail': 0}}}

In [32]:
import collections
lblwithkeywordsH = countDic['Topic']['H']
lblwithkeywordsV = countDic['Topic']['V']
lblwithkeywordsH = collections.Counter(lblwithkeywordsH)
lblwithkeywordsV = collections.Counter(lblwithkeywordsV)
dH = lblwithkeywordsH.most_common(2)
dV = lblwithkeywordsV.most_common(2)

for key,value in dH:
    print(key)
for key,value in dV:
    print(key)    

Search
Authentication
Banking
Automotive


In [33]:
HList=[]
VList=[]
for key in countDic['Topic']['H']:
  if(countDic['Topic']['H'][key]==max(countDic['Topic']['H'].values())):
    HList+=[key]
for key in countDic['Topic']['V']:
  if(countDic['Topic']['V'][key]==max(countDic['Topic']['V'].values())): 
    VList+=[key]

In [34]:
print("This microservice is classified as:",HList,VList)


This microservice is classified as: ['Search', 'Authentication', 'Logging', 'Upload'] ['Banking', 'Automotive']


In [ ]:
max(countDic['Topic']['H'].values())